 <div style="text-align:center;">
  # <span style="color:green; font-size:larger; font-weight:bold;">Vérification de la loi d'Okun dans plusieurs pays du monde</span><br><br>
  <span style="font-weight:bold;">Présenté par:</span><br>
  <span>NOUBOUSSI GNINTEDEM LUCIE MARIMAR</span><br>
  <span>YOUSRA JEDDOUB</span> <br>
  <span>AMINA MANSEUR</span>
</div>


<span style="color:green; font-size:larger; font-weight:bold;">INTRODUCTION</span>

FAIRE UNE INTRODCUTION

# <span style="color:green">I- Importation et installation des packages</span>

In [ ]:
from importlib import reload
import declarations as d
reload(d)

import zipfile
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pycountry
import io
from io import BytesIO
import seaborn as sns
import missingno as msno
import statistics 

# <span style="color:green; ">II- Construction de la base de données</span>

Il faut faire une explication de toutes les étapes que nous allons faire pour constituer notre base 

## <span style="color:green; text-align:center;">II-1 Importation de la base GemDataEXTR.Zip via son url de téléchargement</span>

In [ ]:
url = "https://datacatalogfiles.worldbank.org/ddh-published/0037798/DR0092042/GemDataEXTR.zip?"
d.load(url, "GemDataEXTR.zip")

In [ ]:
# Choix des fichiers à extraire

nom_fichiers = [('Unemployment Rate, seas. adj..xlsx', 'monthly'),
              ('GDP Deflator at Market Prices, LCU.xlsx', 'quarterly')
]

# Extraction des bases dans la mémoire: chomage et PIB déflaté
df_Unemployement, df_GDP = [d.extraire_fichier_zip('GemDataEXTR.zip', nom_fichier, nom_feuille)
                           for nom_fichier, nom_feuille in nom_fichiers]

L'extraction des données sur le taux de chômage et le PIB déflaté par pays est désormais complète. <br>
Dans la prochaine étape, nous procéderons à une exploration rapide de ces données et les fusionnerons pour une analyse plus approfondie.


### <span style="color:green; text-align:center;">II-1-1 Préparation des bases avant fusion</span>

#### <span style="color:green; text-align:center;">II-1-1-1 Base taux de chômage</span>

In [ ]:
# Visualisation
df_Unemployement.head()

In [ ]:
# Suppression des deux prmières lignes
df_Unemployement = df_Unemployement.iloc[2:].copy()

In [ ]:
# Type des données
df_Unemployement.info()


Le type de chaque variable est approprié et correspond aux types attendus.

##### Détection des doublons

In [ ]:
# Vérification des doublons
print("Nombre total de doublons dans df_Unemployement :",
      df_Unemployement[df_Unemployement.duplicated()].shape[0])

##### Correction des noms des pays

In [ ]:
# Appliquer la correction sur chaque colonne du DataFrame
df_Unemployement.columns = d.correct_country_name(df_Unemployement.columns)

# Listes des pays détectés
pays = d.detect_country_name(df_Unemployement.columns)

# Base avec colonnes corrigées
df_Unemployement = df_Unemployement[pays]

In [ ]:
df_Unemployement.head(5)

##### Détection des valeurs manquantes

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_Unemployement)

Les données sont des séries temporelles.<br>
Grâce à la visualisation 2, on constate que pour la plupart des pays,<br>
les valeurs manquantes sont en début de la période considérée.<br>
Ainsi, nous allons garder les pays avec au moins 60% d'observations non manquantes.

In [ ]:
# Suppression des pays avec au moins 40% de valeurs manquantes sur la période
df_Unemployement = d.missing(df_Unemployement)

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_Unemployement)

##### Imputation des valeurs manquantes

In [ ]:
# Imputation des valeurs manquantes 
df_Unemployement = d.fill_missing_with_median(df_Unemployement)

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_Unemployement)

In [ ]:
# Colonnes présentes dans Uemploy
pays = df_Unemployement.columns
print(pays)

In [ ]:
# L'index de la DF
df_Unemployement.index
df_Unemployement.head(5)

##### Transformation des données mensuelles en données trimestrielles

In [ ]:
# Grouper par année de 12 mois chacun
df_Unemployement = df_Unemployement\
    .groupby(df_Unemployement.index.year)\
        .filter(lambda x: len(x) == 12)
df_Unemployement = d.pd.DataFrame(df_Unemployement)

In [ ]:
# Transformation
df_Unemployement = df_Unemployement.resample('Q-JAN').mean()
# Ignorer les jours dans l'index
df_Unemployement.index = df_Unemployement.index.strftime('%Y-%m')

#### <span style="color:green; text-align:center;">II-1-1-2 Base taux de croissance du PIB</span>

In [ ]:
# L'index de la DF
df_GDP.index

In [ ]:
# Formater l'index pour obtenir '1994-01' au lieu de '1994-01-01'
df_GDP.index = df_GDP.index.strftime('%Y-%m')
# Supprimez la ligne avec l'index NaN du DataFrame
df_GDP = df_GDP.drop(df_GDP.index[0])

In [ ]:
# Informations sur la DF (nombre de valeurs non nulles, type de données de chaque colonne...)
df_GDP.info()

##### Correction des noms des pays

In [ ]:
# Appliquer la correction sur chaque colonne du DataFrame
df_GDP.columns = d.correct_country_name(df_GDP.columns)

# Base avec colonnes corrigées presente dans Unemploy
df_GDP=df_GDP[pays]

##### Détection des doublons

In [ ]:
# Vérifier la présence de doublons
print("Nombre total de doublons dans df_GDP :", 
      df_GDP[df_GDP.duplicated()].shape[0])

In [ ]:
# Supprimer les doublons
df_GDP.drop_duplicates(inplace=True)

print("Nombre total de doublons dans df_GDP :", 
      df_GDP[df_GDP.duplicated()].shape[0])

##### Détection des valeurs manquantes

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_GDP)

Les données sont des séries temporelles.<br>
Grâce à la visualisation 2, on constate que pour la plupart des pays,<br>
les valeurs manquantes sont en début de periode d'obervation d'observation.<br>
Ainsi, nous allons garder les pays avec au moins 60% des observations non manquantes.

In [ ]:
# Suppression des pays avec au moins 90% des valeurs manquantes sur la période
df_GDP = d.missing(df_GDP)

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_GDP)

##### Imputation des valeurs manquantes

In [ ]:
# Correction des valeurs manquantes 
df_GDP = d.fill_missing_with_median(df_GDP)

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_GDP)

In [ ]:
# Colonnes présentes dans gdp
pays1 = df_GDP.columns
# Colonnes non présentes dans unemploy et GDP
print(list(set(pays) - set(pays1))) 

### <span style="color:green; text-align:center;">II-1-2 Fusion des deux bases</span>

In [ ]:
reload(d)
## tranformation des bases en format long 
dfs = d.transform(df_Unemployement, 'Unemployment_rate')
dfs1 = d.transform(df_GDP, 'GDP_rate')
dfs1.head()

In [ ]:
# Fusion des bases

df_merge1 = d.pd.merge(dfs, dfs1, on=['YEAR', 'COUNTRY'], how='left')
df_merge1.head()

## <span style="color:green; text-align:center;">II-2 Importation de la base HNP_Stats_EXCEL.Zip via son url de téléchargement</span>

In [ ]:
url = "https://databank.worldbank.org/data/download/HNP_Stats_EXCEL.zip"
d.load(url,"HNP_Stats_EXCEL.zip")

In [ ]:
# Choix des fichiers à extraire
nom_fichiers = [('HNP_StatsEXCEL.xlsx', 'Data')]

# Extraction des bases dans la mémoire: chomage et PIB déflaté
Big_data = [d.extraire_fichier_zip('HNP_Stats_EXCEL.zip', nom_fichier, nom_feuille)
                           for nom_fichier, nom_feuille in nom_fichiers]

L'extraction des données sur le niveau d'éducation, l'espérance de vie, et le taux de croissance de la population. <br>
Dans la prochaine étape, nous procéderons à une exploration rapide de ces données et les fusionnerons pour une analyse plus approfondie.


### <span style="color:green; text-align:center;">II-2-1 Préparation des bases avant fusion</span>

In [ ]:
Big_data = Big_data[0]
Big_data.head()


Le type de chaque variable est approprié et correspond aux types attendus.

#### <span style="color:green; text-align:center;">II-2-1-1 Base espérance de vie</span>

In [ ]:
# Extraction des données sur l'espérance de vie 
df_LE = d.extract2(Big_data, 'expectancy','total')
df_LE.head(2)

In [ ]:
# Construction de la base life expentancy
reload(d)
df_LE = d.treat_info(df_LE, pays1)
df_LE.head(2)

##### Detections des valeurs manquantes

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_LE)

print("Nombre total de valeurs manquantes est de ", 
      d.missing_plot(df_LE))

On n'observe aucune valeur manquante dans la base

#### <span style="color:green; text-align:center;">II-2-1-2 Base taux de croissance démographique</span>

In [ ]:
# Extraction des donnés sur la croissance démographique 
reload(d)
df_pop = d.extract2(Big_data, '^Population growth \(annual %\)$','')
df_pop.head(2)

In [ ]:
# Informations sur la DF (nombre de valeurs non nulles, type de données de chaque colonne...)
df_pop.info()

In [42]:
df_pop.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
338,Africa Eastern and Southern,AFE,Population growth (annual %),SP.POP.GROW,NaN,2.660180,2.732633,2.753248,2.806915,2.840787,...,2.780207,2.774990,2.802586,2.728159,2.655672,2.688371,2.691134,2.678184,2.607472,2.543757
808,Africa Western and Central,AFW,Population growth (annual %),SP.POP.GROW,NaN,2.115789,2.145723,2.190827,2.211360,2.242567,...,2.761839,2.750731,2.723317,2.713059,2.706266,2.669239,2.633982,2.615646,2.573377,2.539799
1278,Arab World,ARB,Population growth (annual %),SP.POP.GROW,NaN,2.571718,2.619388,2.670051,2.701946,2.726862,...,2.299823,2.259226,2.155966,2.109697,2.068739,2.096194,2.062687,1.757899,1.623335,1.788339
1748,Caribbean small states,CSS,Population growth (annual %),SP.POP.GROW,NaN,1.907468,1.794901,1.766298,1.733931,1.673520,...,0.661419,0.632858,0.606569,0.562938,0.528019,0.972338,0.670567,0.278364,0.498457,0.315442
2218,Central Europe and the Baltics,CEB,Population growth (annual %),SP.POP.GROW,NaN,0.909144,0.842174,0.892939,0.933268,0.765652,...,-0.213202,-0.209757,-0.230243,-0.255291,-0.246810,-0.196250,-0.136450,-0.213297,-0.750584,-1.286771


In [43]:
# Construction de la base life expentancy
reload(d)
df_pop = d.treat_info(df_pop, pays1)
df_pop.tail()

Nombre total de doublons dans la base est: 0


Country Code,AUS,AUT,BEL,BGR,CAN,CHE,CZE,DEU,ESP,EST,...,PHL,POL,PRT,ROU,RUS,SGP,SVK,SVN,SWE,USA
2021-01,0.140895,0.435672,0.411602,-0.814846,0.575422,0.765501,-1.811871,0.042327,0.105799,0.105997,...,1.494557,-0.401729,0.626850,-0.746036,0.039793,-4.170336,-0.212359,0.268852,0.600592,0.156747
2021-02,0.140895,0.435672,0.411602,-0.814846,0.575422,0.765501,-1.811871,0.042327,0.105799,0.105997,...,1.494557,-0.401729,0.626850,-0.746036,0.039793,-4.170336,-0.212359,0.268852,0.600592,0.156747
2021-03,0.140895,0.435672,0.411602,-0.814846,0.575422,0.765501,-1.811871,0.042327,0.105799,0.105997,...,1.494557,-0.401729,0.626850,-0.746036,0.039793,-4.170336,-0.212359,0.268852,0.600592,0.156747
2021-04,0.140895,0.435672,0.411602,-0.814846,0.575422,0.765501,-1.811871,0.042327,0.105799,0.105997,...,1.494557,-0.401729,0.626850,-0.746036,0.039793,-4.170336,-0.212359,0.268852,0.600592,0.156747
2022-01,1.238639,0.956288,0.856132,-6.187253,1.823370,0.814796,1.570973,0.720875,0.761702,1.336552,...,1.463316,-2.482061,0.460949,-0.393251,0.073830,3.308621,-0.284861,0.185163,0.680583,0.377565


##### Détection des valeurs manquantes

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_pop)

on observe aucune valeur manquantes dans la base

### <span style="color:green; text-align:center;">II-2-2 Fusion des deux bases</span>

In [ ]:
reload(d)
## Tranformation des bases en format long 
dfs = d.transform(df_LE, 'life_expentancy')
dfs1 = d.transform(df_pop, 'pop_growth_rate')
dfs.head()

In [ ]:
# Fusion des bases

df_merge2 = d.pd.merge(dfs, dfs1, on=['YEAR', 'COUNTRY'], how='left')
df_merge2.head()

In [ ]:
# Fusion merge1 et merge2.

df_merge3 = d.pd.merge(df_merge1, df_merge2, on=['YEAR', 'COUNTRY'], how='left')
df_merge3.head()

In [ ]:
# Enregistrez le DataFrame au format CSV
df_merge3.to_csv('final_data.csv', index=False)

## <span style="color:green; text-align:center;">II-3 Importation de la base Spatial Inequality Database via son url de téléchargement</span>

In [ ]:
url = "https://datacatalogfiles.worldbank.org/ddh-published/0064524/DR0091539/inequality%20GMD%20World%20Bank.xlsx?versionId=2023-05-22T17:13:22.2930786Z"
d.load(url,"inequality GMD World Bank.xlsx")

In [ ]:
df_SID = d.pd.read_excel("inequality GMD World Bank.xlsx",  sheet_name='data', index_col=0)
df_SID.head()

### <span style="color:green; text-align:center;">II-3-1 Préparation des bases avant fusion</span>

#### <span style="color:green; text-align:center;">II-3-1-1 base indice Theil</span>

In [ ]:
# Base de données de l'indice Theil
df_theil = df_SID.pivot(index='year', columns='countryname', values='index')
df_theil.head(5)

In [ ]:
df_theil = df_theil.drop(columns=df_theil.columns[0])
df_theil.info()

In [ ]:
#df_theil.columns

##### Corrections des noms des pays

In [ ]:
# Appliquer la correction sur chaque colonne du DataFrame
df_theil.columns = d.correct_country_name(df_theil.columns)

# base avec colonnes corrigées contenues dans les autres bases
#df_theil=df_theil[pays1]
#on a que 19 pays

In [ ]:
# Toutes les quatres bases à extraire ici
# La moyenne de l'indice de theil sur toutes les périodes
mean_theil = theil.mean(axis=0) 
# Base de données du taux d'urbanisation
urban = gmd.pivot(index='year', columns='countryname', values='sp_urb_totl_in_zs')
urban.head(5)
# La moyenne du taux d'urbanisation sur toutes les périodes
mean_urban = urban.mean(axis=0)
# Base de données du ratio de pauvreté
poverty = gmd.pivot(index='year', columns='countryname', values='si_pov_lmic')
poverty.head(5)
# La moyenne du ratio de pauvreté sur toutes les périodes
mean_poverty = poverty.mean(axis=0)

# IV- GMD  ici on a pas les meme années que faire ??

## 1.  Importation

In [ ]:
# Gini Mean Difference
gmd = pd.read_excel("C:/Users/yousr/Downloads/Projet_py/inequality GMD World Bank.xlsx",  sheet_name='data', index_col=0)
gmd.head(5)
col_names = gmd.columns
print(col_names)

## 2. Construction de la DF

### 2.1. Extraction des colonnes nécessaires

In [ ]:
# Base de données de l'indice Theil
theil = gmd.pivot(index='year', columns='countryname', values='index')
theil.head(5)

In [ ]:
# La moyenne de l'indice de theil sur toutes les périodes
mean_theil = theil.mean(axis=0) 

In [ ]:
# Base de données du taux d'urbanisation
urban = gmd.pivot(index='year', columns='countryname', values='sp_urb_totl_in_zs')
urban.head(5)

In [ ]:
# La moyenne du taux d'urbanisation sur toutes les périodes
mean_urban = urban.mean(axis=0)

In [ ]:
# Base de données du ratio de pauvreté
poverty = gmd.pivot(index='year', columns='countryname', values='si_pov_lmic')
poverty.head(5)

In [ ]:
# La moyenne du ratio de pauvreté sur toutes les périodes
mean_poverty = poverty.mean(axis=0)

In [ ]:
# Dans pandas, une Df à une colonne correspond à une série. On donne un nom à chaque série pour une éventuelle jointure
mean_poverty.name = 'Ratio de pauvreté'
mean_urban.name = "Taux urbanisation"
mean_theil.name = 'Theil'

## 2.2. Jointure 

In [ ]:
# Jointure entre df1 et df2 sur la colonne "countryname"
merged1 = pd.merge(mean_poverty, mean_urban, on='countryname', how='inner')
# Jointure entre le résultat précédent (merged_df) et df3 sur la colonne "countryname"
final_merged1 = pd.merge(merged1, mean_theil, on='countryname', how='inner')

In [ ]:
# Dans pandas, une Df à une colonne correspond à une série. On donne un nom à chaque série pour une éventuelle jointure
mean_LE.name = 'Espérance de vie'
mean_PGR.name = "Croissance démo"

In [ ]:
final_merged1.index.name = 'Country Name'  ####################################################################################
mean_LE.index.name = 'Country Name'
mean_PGR.index.name = 'Country Name'
merged_mean.index.name = 'Country Name'

In [ ]:
# Jointure entre df1 et df2 sur la colonne "countryname"
merged2 = pd.merge(final_merged1, mean_PGR, on='Country Name', how='inner')
# Jointure entre le résultat précédent (merged_df) et df3 sur la colonne "countryname"
merged3 = pd.merge(merged2, mean_LE, on='Country Name', how='inner')

# Transposé de merged3

In [ ]:
merged3_transpose = merged3.transpose()
col_names = merged3_transpose.columns
merged3_transpose.head(5)

# Code ISO des pays

In [ ]:
# Pour faciliter la lecture de la DF, on remplace les noms des pays par leurs codes ISO  correspondant

# Dictionnaire de correspondance entre noms complets des pays et leurs codes
corresp = {country.name: country.alpha_3 for country in pycountry.countries}

# Liste initiale des noms complets des colonnes
country = col_names
# Transformation des noms complets des colonnes en abréviations
country_codes = [corresp.get(pays, pays) for pays in country]

# Remplacement des noms des pays par leurs codes dans la merged
merged3_transpose.columns = country_codes
merged3_transpose.columns
merged3_transpose.head(5)

#Ordre alphabétique des colonnes 
merged3_transpose_sort = merged3_transpose.sort_index(axis=1)
merged3_transpose_sort.head(5)

In [ ]:
merged4 = merged3_transpose.transpose()
merged4.index.name = 'Country Name'
merged4.head(5)

In [ ]:
final_merged = pd.merge(merged4, merged_mean, on='Country Name', how='inner')